In [1]:
import pandas as pd

from plots import plot_confusion_matrix, plot_roc_curve

In [2]:
import mlflow

# Para que funcione, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [3]:
!echo $AWS_ACCESS_KEY_ID
!echo $AWS_SECRET_ACCESS_KEY
!echo $MLFLOW_S3_ENDPOINT_URL

minio
minio123
http://localhost:9000


# MLFlow

## MLFlow Tracking

### Más conceptos sobre MLFlow Tracking

En esta notebook, haremos uso de otra característica muy útil: `mlflow.autolog()`. Esta función registra automáticamente información esencial durante el entrenamiento y la evaluación del modelo, evitando la necesidad de especificar manualmente qué elementos rastrear. Está implementada para varias de las principales librerías de machine learning, y la utilizaremos específicamente con Scikit-Learn.

Entre las cosas que `autolog()` registra automáticamente se encuentran:

- **Parámetros**: Los hiperparámetros utilizados para entrenar el modelo (por ejemplo, n_estimators, max_depth en un RandomForest).
- **Métricas**: Métricas de evaluación calculadas durante el entrenamiento o el testeo (por ejemplo, accuracy, precision, recall, F1-score, RMSE, R2).
- **Modelos**: El modelo entrenado en sí, lo que permite cargarlo y reutilizarlo fácilmente más adelante para hacer predicciones.
- **Artefactos**: Otros archivos relevantes, como gráficos (por ejemplo, curvas ROC, matrices de confusión) o resúmenes de datos importantes.
- **Código fuente**: El script que ejecutó el entrenamiento, lo cual facilita la reproducibilidad.
- **Entorno**: Información sobre el entorno de software, incluidas las versiones de las librerías, para ayudar a reproducir los resultados.

Se utiliza de la siguiente manera, y debe invocarse antes de comenzar el entrenamiento:

```python
mlflow.sklearn.autolog()
```

### Uso de MLFlow con un ejemplo: Iris Dataset y Random Forest

Vamos a emplear MLFlow para entrenar dos modelos utilizando el conjunto de datos Iris. Los modelos serán Logistic Regression y Random Forest, ambos de la librería Scikit-Learn. En esta notebook, nos enfocaremos en el modelo de Random Forest.

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, accuracy_score, recall_score

In [5]:
mlflow.set_tracking_uri('http://localhost:5001')

In [6]:
experiment_name = "experiment_iris"

if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(name=experiment_name, 
                             tags={"project":"iris-dataset", 
                                   "team": "mlops1-fiuba"}) 

experiment = mlflow.get_experiment_by_name(experiment_name)

In [7]:
# Cargamos los datos
data = load_iris()

# Separamos entre evaluación y testeo
X_train, X_test, y_train, y_test = train_test_split(data['data'][:, :2], data['target'], test_size=0.2, random_state=42)

### Búsqueda de hiperparámetros

Aquí realizaremos la búsqueda de hiperparámetros para optimizar el rendimiento del modelo.

In [8]:
# Habilitamos el autologging para scikit-learn
mlflow.sklearn.autolog()

run_name = "random_forest_hyper_search"

2025/07/12 19:27:20 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.6.1, but the installed version is 1.7.0. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


In [9]:
# Iniciamos un run de MLflow
with mlflow.start_run(experiment_id = experiment.experiment_id, 
                      run_name=run_name,
                      tags={"model":"random_forest"}):
    model = RandomForestClassifier()

    # Definimos los hiperparámetros para la búsqueda
    grid = {
        'max_depth':[6,8,10], 
        'min_samples_split':[2,3,4,5],
        'min_samples_leaf':[2,3,4,5],
        'max_features': [2,3]
        }

    # Hacemos la búsqueda
    iris_grid = GridSearchCV(model, grid, cv=5) 
    iris_grid_results = iris_grid.fit(X_train, y_train)

    print(f'Los mejores parámetros son: {iris_grid_results.best_params_}')

2025/07/12 19:27:49 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/07/12 19:27:51 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/07/12 19:27:51 INFO mlflow.sklearn.utils: Logging the 5 best runs, 91 runs will be omitted.


🏃 View run rambunctious-cat-356 at: http://localhost:5001/#/experiments/2/runs/da9e14894c2b40caa9960fce814dfac3
🧪 View experiment at: http://localhost:5001/#/experiments/2
🏃 View run capable-bear-816 at: http://localhost:5001/#/experiments/2/runs/c9fb920fb5f641cbadf742113b71f330
🧪 View experiment at: http://localhost:5001/#/experiments/2
🏃 View run trusting-ram-323 at: http://localhost:5001/#/experiments/2/runs/a27bea803b09464c8057aa3f5ce315e6
🧪 View experiment at: http://localhost:5001/#/experiments/2
🏃 View run worried-colt-569 at: http://localhost:5001/#/experiments/2/runs/873b5703ebba4fe283a60c137b14346b
🧪 View experiment at: http://localhost:5001/#/experiments/2
🏃 View run bittersweet-wolf-355 at: http://localhost:5001/#/experiments/2/runs/64e397b8caf846bbb48bd79cddf6768c
🧪 View experiment at: http://localhost:5001/#/experiments/2
Los mejores parámetros son: {'max_depth': 6, 'max_features': 2, 'min_samples_leaf': 4, 'min_samples_split': 4}
🏃 View run random_forest_hyper_search at:

### Entrenamiento del modelo Random Forest

Una vez realizada la búsqueda de hiperparámetros, registramos los datos del mejor modelo encontrado.

In [10]:
# Habilitamos el autologging para scikit-learn
run_name = "random_forest_training"

mlflow.sklearn.autolog()

2025/07/12 19:27:51 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.6.1, but the installed version is 1.7.0. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


In [11]:
# Iniciamos un run de MLflow
with mlflow.start_run(experiment_id = experiment.experiment_id, 
                      run_name=run_name,
                      tags={"model":"random_forest"}):
    
    # Usamos los mejores parámetros
    model = RandomForestClassifier(**iris_grid_results.best_params_)

    # Entrenamos el modelo
    model.fit(X_train, y_train)

    # Obtenemos la prediccion del set de evaluación
    y_pred = model.predict(X_test)

    # Se calculan las métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')

    # Como artefactos, obtenemos las gráficas de la curva ROC y la matriz de confusion
    matrix_plot = plot_confusion_matrix(y_test, y_pred, save_path=None)
    roc_plots = plot_roc_curve(y_test, model.predict_proba(X_test), save_path=None)

    # Podemos loggear lo que autolog no hace automaticomente
    metrics ={
        'accuracy': accuracy,
        'precision': precision, 
        'recall': recall 
        }
    
    mlflow.log_metrics(metrics)

    matrix_plot = plot_confusion_matrix(y_test, y_pred, save_path=None)
    roc_plots = plot_roc_curve(y_test, model.predict_proba(X_test), save_path=None)

    mlflow.log_figure(matrix_plot, artifact_file="matrix_plot.png")
    mlflow.log_figure(roc_plots[0], artifact_file="roc_curve_1_plot.png")
    mlflow.log_figure(roc_plots[1], artifact_file="roc_curve_2_plot.png")
    mlflow.log_figure(roc_plots[2], artifact_file="roc_curve_3_plot.png")

2025/07/12 19:27:54 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Accuracy: 0.7
Precision: 0.7277777777777777
Recall: 0.7
🏃 View run random_forest_training at: http://localhost:5001/#/experiments/2/runs/38d33946b2ab4209a70a886e250364e8
🧪 View experiment at: http://localhost:5001/#/experiments/2


#### Realizar predicciones con un modelo de MLFlow

Para obtener predicciones, debemos ir al apartado de *Artifacts*, donde MLFlow indica cómo realizar inferencias con el modelo, ya sea utilizando Spark o Python. Copiamos ese fragmento de código y lo ejecutamos, proporcionando los datos que deseamos predecir.

In [12]:
runs_df = mlflow.search_runs(
            experiment_ids=experiment.experiment_id,
            filter_string=f"tags.mlflow.runName = '{run_name}'"   
        )

run_id = runs_df["run_id"].iloc[0]
model_uri = f"runs:/{runs_df['run_id'].iloc[0]}/model" 

# Cargamos el modelo usando el modulo de scikit-learn
loaded_model = mlflow.sklearn.load_model(model_uri)

# Predecimos usando un dataframe de Pandas
loaded_model.predict(pd.DataFrame(X_test))

array([1, 1, 2, 1, 2, 0, 1, 2, 2, 2, 2, 0, 1, 0, 0, 2, 2, 1, 1, 2, 0, 1,
       0, 2, 2, 1, 1, 2, 0, 0])